# Validación de datos SEGCOVID

## Carga de los datos

Este jupyter notebook esta generando los datos de un año en especifico (2012 el parquet part.0.parquet de SEGCOVID) para coincidir con las tablas compartidas del  power bi (Tablas agregadas) 

In [50]:
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.appName("GeneraciónTablasSEGCOVID").getOrCreate()

In [51]:
#spark.stop()

In [52]:
# Leer los archivos Parquet de un año 
df1_parquet = spark.read.parquet("/rawdata/segcovid/segcovid_parquet_2022/part.0.parquet")

df=df1_parquet

In [53]:
# Contar el número de registros en el DataFrame unido
total_registros = df.count()

# Listar las columnas del DataFrame unido
columnas = df.columns

# Mostrar resultados
print(f"Total de registros en el DataFrame unido: {total_registros}")
print(f"Columnas del DataFrame unido: {columnas}")

Total de registros en el DataFrame unido: 128343
Columnas del DataFrame unido: ['PersonaBasicaID', 'AntecedenteVIH', 'AntecedenteHipertension', 'AntecedenteDiabetes', 'AntecedenteEnfermedadHuerfana', 'AntecedenteCancer', 'AntecedenteArtritis', 'AntecedenteERC', 'AntecedenteEPOC', 'AntecedenteAsma', 'AntecedenteObesidad', 'FactorRiesgoMayor59', 'FactorRiesgoDiabetes', 'FactorRiesgoEnfermedadCardiovascular', 'FactorRiesgoEnfermedadRespiratoriaCronica', 'FactorRiesgoCancer', 'FactorRiesgoInmunodeficiencia', 'FactorRiesgoCondicionImpidaAislamiento', 'FechaIngresoAtencion', 'FechaEgresoAtencion', 'FechaRegistro', 'FechaSeguimiento', 'DepartamentoAtencion', 'MunicipioAtencion', 'AmbitoAtencion', 'SignosAlarmaRespiracionRapida', 'SignosAlarmaFiebre', 'SignosAlarmaPechoSuena', 'SignosAlarmaSomnolencia', 'SignosAlarmaConvulsiones', 'SignosAlarmaDecaimiento', 'SignosAlarmaDeterioro', 'SintomasFiebre', 'SintomasTos', 'SintomasFatiga', 'SintomasDificultadRespirar', 'SintomasDolorGarganta', 'Sintom

In [54]:
# Contar el número de registros - me da satisfactoriamente la cantidad de regIstros del año 2022 (128.343)
record_count = df.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 128343


In [55]:
# Mostrar el esquema (columnas y tipos de datos)
df.printSchema()

# Listar las columnas
print(df.columns)

root
 |-- PersonaBasicaID: string (nullable = true)
 |-- AntecedenteVIH: string (nullable = true)
 |-- AntecedenteHipertension: string (nullable = true)
 |-- AntecedenteDiabetes: string (nullable = true)
 |-- AntecedenteEnfermedadHuerfana: string (nullable = true)
 |-- AntecedenteCancer: string (nullable = true)
 |-- AntecedenteArtritis: string (nullable = true)
 |-- AntecedenteERC: string (nullable = true)
 |-- AntecedenteEPOC: string (nullable = true)
 |-- AntecedenteAsma: string (nullable = true)
 |-- AntecedenteObesidad: string (nullable = true)
 |-- FactorRiesgoMayor59: string (nullable = true)
 |-- FactorRiesgoDiabetes: string (nullable = true)
 |-- FactorRiesgoEnfermedadCardiovascular: string (nullable = true)
 |-- FactorRiesgoEnfermedadRespiratoriaCronica: string (nullable = true)
 |-- FactorRiesgoCancer: string (nullable = true)
 |-- FactorRiesgoInmunodeficiencia: string (nullable = true)
 |-- FactorRiesgoCondicionImpidaAislamiento: string (nullable = true)
 |-- FechaIngresoAt

# Empieza el proceso de generación de los datos agrupados por atenciones

In [56]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Crear una sesión de Spark
#spark = SparkSession.builder.appName("DataFrame Example").getOrCreate()


# Cargar el archivo Parquet en un DataFrame
#df = df1

# Seleccionar solo las columnas deseadas (sin 'personaid_count' ya que no existe aún)
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 'AntecedenteHipertension',
    'AntecedenteArtritis', 'AntecedenteAsma', 'AntecedenteCancer', 'AntecedenteDiabetes', 'AntecedenteEnfermedadHuerfana',
    'AntecedenteEPOC', 'AntecedenteERC', 'AntecedenteObesidad', 'AntecedenteVIH',
    
    'FactorRiesgoCancer','FactorRiesgoCondicionImpidaAislamiento' ,'FactorRiesgoDiabetes' ,'FactorRiesgoEnfermedadCardiovascular',
    'FactorRiesgoEnfermedadRespiratoriaCronica', 'FactorRiesgoInmunodeficiencia', 'FactorRiesgoMayor59' 
   
]
df_filtered = df.select(*columns_to_keep)

# Agregar una nueva columna con el recuento distintivo de 'personaid'
window_spec = Window.partitionBy('PersonaBasicaID')
df_filtered = df_filtered.withColumn('personaid_count', F.count('PersonaBasicaID').over(window_spec))

# Mostrar las primeras filas del DataFrame filtrado para verificar
df_filtered.show()

# Mostrar los nombres de las columnas filtradas para verificar
print("Nombres de las columnas filtradas:")
print(df_filtered.columns)

# Agrupar por la columna 'PersonaBasicaID'
grouped_df = df_filtered.groupBy('PersonaBasicaID').agg(
    F.first('FechaRegistro').alias('FechaRegistro'),
    F.first('MunicipioAtencion').alias('MunicipioAtencion'),
    F.first('AntecedenteHipertension').alias('AntecedenteHipertension'),
    F.first('AntecedenteArtritis').alias('AntecedenteArtritis'),
    F.first('AntecedenteAsma').alias('AntecedenteAsma'),
    F.first('AntecedenteCancer').alias('AntecedenteCancer'),
    F.first('AntecedenteDiabetes').alias('AntecedenteDiabetes'),
    F.first('AntecedenteEnfermedadHuerfana').alias('AntecedenteEnfermedadHuerfana'),
    F.first('AntecedenteEPOC').alias('AntecedenteEPOC'),
    F.first('AntecedenteERC').alias('AntecedenteERC'),
    F.first('AntecedenteObesidad').alias('AntecedenteObesidad'),
    F.first('AntecedenteVIH').alias('AntecedenteVIH'),
    
    F.first('FactorRiesgoCancer').alias('FactorRiesgoCancer'),
    F.first('FactorRiesgoCondicionImpidaAislamiento').alias('FactorRiesgoCondicionImpidaAislamiento'),
    F.first('FactorRiesgoDiabetes').alias('FactorRiesgoDiabetes'),
    F.first('FactorRiesgoEnfermedadCardiovascular').alias('FactorRiesgoEnfermedadCardiovascular'),
    F.first('FactorRiesgoEnfermedadRespiratoriaCronica').alias('FactorRiesgoEnfermedadRespiratoriaCronica'),
    F.first('FactorRiesgoInmunodeficiencia').alias('FactorRiesgoInmunodeficiencia'),
    F.first('FactorRiesgoMayor59').alias('FactorRiesgoMayor59'),
    
    F.first('personaid_count').alias('personaid_count')
)
# Mostrar las primeras filas del DataFrame agrupado para verificar
grouped_df.show()


+---------------+-------------+-------------------+-----------------------+-------------------+---------------+-----------------+-------------------+-----------------------------+---------------+--------------+-------------------+--------------+------------------+--------------------------------------+--------------------+------------------------------------+-----------------------------------------+-----------------------------+-------------------+---------------+
|PersonaBasicaID|FechaRegistro|  MunicipioAtencion|AntecedenteHipertension|AntecedenteArtritis|AntecedenteAsma|AntecedenteCancer|AntecedenteDiabetes|AntecedenteEnfermedadHuerfana|AntecedenteEPOC|AntecedenteERC|AntecedenteObesidad|AntecedenteVIH|FactorRiesgoCancer|FactorRiesgoCondicionImpidaAislamiento|FactorRiesgoDiabetes|FactorRiesgoEnfermedadCardiovascular|FactorRiesgoEnfermedadRespiratoriaCronica|FactorRiesgoInmunodeficiencia|FactorRiesgoMayor59|personaid_count|
+---------------+-------------+-------------------+---------

+---------------+-------------+-------------------+-----------------------+-------------------+---------------+-----------------+-------------------+-----------------------------+---------------+--------------+-------------------+--------------+------------------+--------------------------------------+--------------------+------------------------------------+-----------------------------------------+-----------------------------+-------------------+---------------+
|PersonaBasicaID|FechaRegistro|  MunicipioAtencion|AntecedenteHipertension|AntecedenteArtritis|AntecedenteAsma|AntecedenteCancer|AntecedenteDiabetes|AntecedenteEnfermedadHuerfana|AntecedenteEPOC|AntecedenteERC|AntecedenteObesidad|AntecedenteVIH|FactorRiesgoCancer|FactorRiesgoCondicionImpidaAislamiento|FactorRiesgoDiabetes|FactorRiesgoEnfermedadCardiovascular|FactorRiesgoEnfermedadRespiratoriaCronica|FactorRiesgoInmunodeficiencia|FactorRiesgoMayor59|personaid_count|
+---------------+-------------+-------------------+---------

In [57]:
# Contar el número de registros - me da satisfactoriamente la cantidad de registros en cada año ( 97.416)
record_count = grouped_df.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 97416


In [58]:
 df_final =grouped_df


In [59]:
# Contar el número de registros - me da satisfactoriamente la cantidad de regustros en cada año 
record_count = df_final.count()

# Mostrar el número de registros
print(f"El número de registros en el archivo Parquet es: {record_count}")

El número de registros en el archivo Parquet es: 97416


In [60]:
# Mostrar las primeras filas del DataFrame agrupado para verificar
df_final.show()

+---------------+-------------+-------------------+-----------------------+-------------------+---------------+-----------------+-------------------+-----------------------------+---------------+--------------+-------------------+--------------+------------------+--------------------------------------+--------------------+------------------------------------+-----------------------------------------+-----------------------------+-------------------+---------------+
|PersonaBasicaID|FechaRegistro|  MunicipioAtencion|AntecedenteHipertension|AntecedenteArtritis|AntecedenteAsma|AntecedenteCancer|AntecedenteDiabetes|AntecedenteEnfermedadHuerfana|AntecedenteEPOC|AntecedenteERC|AntecedenteObesidad|AntecedenteVIH|FactorRiesgoCancer|FactorRiesgoCondicionImpidaAislamiento|FactorRiesgoDiabetes|FactorRiesgoEnfermedadCardiovascular|FactorRiesgoEnfermedadRespiratoriaCronica|FactorRiesgoInmunodeficiencia|FactorRiesgoMayor59|personaid_count|
+---------------+-------------+-------------------+---------

# Empieza el proceso de generación de tablas SEGCOVID

# ANTECEDENTES (Pestaña recibida en PowerBI)

## 1. Agrupado por año, mes, AntecedenteHipertension y MunicipioAtencion

In [61]:
# Cargar el archivo Parquet en un DataFrame ( uno distinto por si deseo comparar)
df2 = df_final

In [62]:
# Selecciona la columna 'FechaRegistro' y muestra sus datos
#df.select('FechaRegistro').show(100, truncate=False)

In [63]:

# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 'AntecedenteHipertension',
    'AntecedenteArtritis', 'AntecedenteAsma', 'AntecedenteCancer', 'AntecedenteDiabetes', 'AntecedenteEnfermedadHuerfana',
    'AntecedenteEPOC', 'AntecedenteERC', 'AntecedenteObesidad', 'AntecedenteVIH', 'personaid_count' 
   
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y MunicipioAtencion
result_df = df_filtered.groupBy('Año', 'Mes','AntecedenteHipertension','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-----------------------+--------------------+------------------+--------------------+
| Año|Mes|AntecedenteHipertension|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------------------+--------------------+------------------+--------------------+
|2022|  4|                     No|       25785 - Tabio|                 1|                   3|
|2022|  4|                     No|    -1 - No Definido|               349|                 355|
|2022|  6|                     No| 68001 - Bucaramanga|                65|                  80|
|2022|  8|                     Sí|    05001 - Medellín|                36|                  36|
|2022|  6|                     No|     15238 - Duitama|                 8|                  13|
|2022|  7|                     Sí| 68547 - Piedecuesta|                 1|                   1|
|2022|  7|                     No|      68770 - Suaita|                 1|                   2|
|2022|  7|                     No|      

Se compara los datos con el power BI dando satisfactorio los resultados
Se valida correctamente: 
Año	Mes	AntecedenteHipertension	MunicipioAtencion	# Personas	# Atenciones
2022	enero	No	                05129 - Caldas	178               	231

2022	enero	No	               05308 - Girardota	93	            104

# 2. Agrupado por año, mes , AntecedenteArtritis y MunicipioAtencion

In [64]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 'AntecedenteHipertension',
    'AntecedenteArtritis', 'AntecedenteAsma', 'AntecedenteCancer', 'AntecedenteDiabetes', 'AntecedenteEnfermedadHuerfana',
    'AntecedenteEPOC', 'AntecedenteERC', 'AntecedenteObesidad', 'AntecedenteVIH', 'personaid_count' 
   
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y AntecedenteArtritis
result_df = df_filtered.groupBy('Año', 'Mes','AntecedenteArtritis','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)


+----+---+-------------------+--------------------+------------------+--------------------+
| Año|Mes|AntecedenteArtritis|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+-------------------+--------------------+------------------+--------------------+
|2022| 10|                 No|    54518 - Pamplona|                 1|                   1|
|2022|  4|                 No|       25785 - Tabio|                 1|                   3|
|2022|  4|                 No|    -1 - No Definido|               372|                 379|
|2022|  6|                 No| 68001 - Bucaramanga|                72|                  91|
|2022|  6|                 No|     15238 - Duitama|                 8|                  13|
|2022|  6|                 No|23189 - Ciénaga D...|                 1|                   3|
|2022|  7|                 No|      68770 - Suaita|                 1|                   2|
|2022|  7|                 No|       52001 - Pasto|                76|          

Se compara los datos con el power BI dando satisfactorio los resultados
Se valida correctamente: 

Año	Mes	AntecedenteArtritis	MunicipioAtencion	# Personas	# Atenciones
2022	enero	No	17873 - Villamaría	10	10

2022	enero	No	05212 - Copacabana	126	151

# 3. Agrupado por año, mes , AntecedenteAsma y MunicipioAtencion

In [65]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 'AntecedenteHipertension',
    'AntecedenteArtritis', 'AntecedenteAsma', 'AntecedenteCancer', 'AntecedenteDiabetes', 'AntecedenteEnfermedadHuerfana',
    'AntecedenteEPOC', 'AntecedenteERC', 'AntecedenteObesidad', 'AntecedenteVIH', 'personaid_count' 
   
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y AntecedenteAsma
result_df = df_filtered.groupBy('Año', 'Mes','AntecedenteAsma','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)


+----+---+---------------+--------------------+------------------+--------------------+
| Año|Mes|AntecedenteAsma|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+---------------+--------------------+------------------+--------------------+
|2022|  6|             No| 68001 - Bucaramanga|                71|                  84|
|2022|  4|             No|    -1 - No Definido|               361|                 368|
|2022|  6|             No|     15238 - Duitama|                 8|                  13|
|2022|  9|             No|    -1 - No Definido|               324|                 325|
|2022|  7|             No|       52001 - Pasto|                74|                  93|
|2022| 11|             No| 11001 - Bogotá D.C.|                75|                 154|
|2022|  2|             No|     05736 - Segovia|                 2|                   2|
|2022|  1|             No|   05308 - Girardota|               103|                 123|
|2022| 10|             No|99001 

# 4. Agrupado por año, mes, AntecedenteCancer y MunicipioAtencion

In [66]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 'AntecedenteHipertension',
    'AntecedenteArtritis', 'AntecedenteAsma', 'AntecedenteCancer', 'AntecedenteDiabetes', 'AntecedenteEnfermedadHuerfana',
    'AntecedenteEPOC', 'AntecedenteERC', 'AntecedenteObesidad', 'AntecedenteVIH', 'personaid_count' 
   
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y AntecedenteCancer
result_df = df_filtered.groupBy('Año', 'Mes','AntecedenteCancer','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-----------------+--------------------+------------------+--------------------+
| Año|Mes|AntecedenteCancer|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------------+--------------------+------------------+--------------------+
|2022| 10|               No|    54518 - Pamplona|                 1|                   1|
|2022|  4|               No|       25785 - Tabio|                 1|                   3|
|2022|  4|               No|    -1 - No Definido|               369|                 376|
|2022|  6|               No| 68001 - Bucaramanga|                70|                  87|
|2022|  6|               No|     15238 - Duitama|                 8|                  13|
|2022|  8|               Sí|    05001 - Medellín|                 7|                   7|
|2022|  6|               No|23189 - Ciénaga D...|                 1|                   3|
|2022|  7|               No|      68770 - Suaita|                 1|                   2|
|2022|  7|

# 5. Agrupado por año, mes, AntecedenteDiabetes y MunicipioAtencion

In [67]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 'AntecedenteHipertension',
    'AntecedenteArtritis', 'AntecedenteAsma', 'AntecedenteCancer', 'AntecedenteDiabetes', 'AntecedenteEnfermedadHuerfana',
    'AntecedenteEPOC', 'AntecedenteERC', 'AntecedenteObesidad', 'AntecedenteVIH', 'personaid_count' 
   
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y AntecedenteDiabetes
result_df = df_filtered.groupBy('Año', 'Mes','AntecedenteDiabetes','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-------------------+--------------------+------------------+--------------------+
| Año|Mes|AntecedenteDiabetes|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+-------------------+--------------------+------------------+--------------------+
|2022| 10|                 No|    54518 - Pamplona|                 1|                   1|
|2022|  4|                 No|       25785 - Tabio|                 1|                   3|
|2022|  4|                 No|    -1 - No Definido|               371|                 377|
|2022|  6|                 No| 68001 - Bucaramanga|                69|                  87|
|2022|  8|                 Sí|    05001 - Medellín|                 9|                   9|
|2022|  6|                 No|     15238 - Duitama|                 8|                  13|
|2022|  6|                 No|23189 - Ciénaga D...|                 1|                   3|
|2022|  7|                 No|      68770 - Suaita|                 1|          

# 6. Agrupado por año, mes, AntecedenteEnfermedadHuerfana y MunicipioAtencion

In [68]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 'AntecedenteHipertension',
    'AntecedenteArtritis', 'AntecedenteAsma', 'AntecedenteCancer', 'AntecedenteDiabetes', 'AntecedenteEnfermedadHuerfana',
    'AntecedenteEPOC', 'AntecedenteERC', 'AntecedenteObesidad', 'AntecedenteVIH', 'personaid_count' 
   
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y AntecedenteEnfermedadHuerfana
result_df = df_filtered.groupBy('Año', 'Mes','AntecedenteEnfermedadHuerfana','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-----------------------------+--------------------+------------------+--------------------+
| Año|Mes|AntecedenteEnfermedadHuerfana|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------------------------+--------------------+------------------+--------------------+
|2022|  6|                           No| 68001 - Bucaramanga|                73|                  92|
|2022|  4|                           No|    -1 - No Definido|               372|                 379|
|2022|  8|                           Sí|    05001 - Medellín|                 3|                   3|
|2022|  6|                           No|     15238 - Duitama|                 8|                  13|
|2022|  9|                           No|    -1 - No Definido|               333|                 335|
|2022|  7|                           No|       52001 - Pasto|                76|                  95|
|2022| 11|                           No| 11001 - Bogotá D.C.|                77|  

# 7. Agrupado por año, mes , AntecedenteEPOC y MunicipioAtencion

In [69]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 'AntecedenteHipertension',
    'AntecedenteArtritis', 'AntecedenteAsma', 'AntecedenteCancer', 'AntecedenteDiabetes', 'AntecedenteEnfermedadHuerfana',
    'AntecedenteEPOC', 'AntecedenteERC', 'AntecedenteObesidad', 'AntecedenteVIH', 'personaid_count' 
   
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y AntecedenteEPOC
result_df = df_filtered.groupBy('Año', 'Mes','AntecedenteEPOC','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+---------------+--------------------+------------------+--------------------+
| Año|Mes|AntecedenteEPOC|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+---------------+--------------------+------------------+--------------------+
|2022|  6|             No| 68001 - Bucaramanga|                68|                  83|
|2022|  4|             No|    -1 - No Definido|               369|                 376|
|2022|  9|             No|    -1 - No Definido|               316|                 318|
|2022|  7|             No|       52001 - Pasto|                72|                  91|
|2022| 11|             No| 11001 - Bogotá D.C.|                71|                 144|
|2022|  1|             No|   05308 - Girardota|               106|                 128|
|2022| 10|             No|99001 - Puerto Ca...|                 2|                   2|
|2022| 10|             No|       85001 - Yopal|                 2|                   2|
|2022|  3|             No|     8

se verifican algunos resultados de enero, coinciden perfectamente

Año	Mes	AntecedenteEPOC	MunicipioAtencion	# Personas	# Atenciones
2022	enero	No	05129 - Caldas	193	254

Año	Mes	AntecedenteEPOC	MunicipioAtencion	# Personas	# Atenciones
2022	enero	No	05308 - Girardota	106	128

# 8. Agrupado por año, mes, AntecedenteERC y MunicipioAtencion

In [70]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 'AntecedenteHipertension',
    'AntecedenteArtritis', 'AntecedenteAsma', 'AntecedenteCancer', 'AntecedenteDiabetes', 'AntecedenteEnfermedadHuerfana',
    'AntecedenteEPOC', 'AntecedenteERC', 'AntecedenteObesidad', 'AntecedenteVIH', 'personaid_count' 
   
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y AntecedenteERC
result_df = df_filtered.groupBy('Año', 'Mes','AntecedenteERC','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+--------------+--------------------+------------------+--------------------+
| Año|Mes|AntecedenteERC|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+--------------+--------------------+------------------+--------------------+
|2022|  6|            No| 68001 - Bucaramanga|                71|                  90|
|2022|  4|            No|    -1 - No Definido|               368|                 374|
|2022|  6|            No|     15238 - Duitama|                 8|                  13|
|2022|  9|            No|    -1 - No Definido|               324|                 326|
|2022|  7|            No|       52001 - Pasto|                72|                  91|
|2022| 11|            No| 11001 - Bogotá D.C.|                68|                 135|
|2022|  2|            No|     05736 - Segovia|                 2|                   2|
|2022|  1|            No|   05308 - Girardota|               106|                 124|
|2022| 10|            No|99001 - Puerto Ca.

da correctamente al verificar unos registros de enero

Año	Mes	AntecedenteERC	MunicipioAtencion	# Personas	# Atenciones
2022	enero	No	05129 - Caldas	195	256

Año	Mes	AntecedenteERC	MunicipioAtencion	# Personas	# Atenciones
2022	enero	No	05308 - Girardota	106	124

# 9. Agrupado por año, mes, AntecedenteObesidad y MunicipioAtencion

In [71]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 'AntecedenteHipertension',
    'AntecedenteArtritis', 'AntecedenteAsma', 'AntecedenteCancer', 'AntecedenteDiabetes', 'AntecedenteEnfermedadHuerfana',
    'AntecedenteEPOC', 'AntecedenteERC', 'AntecedenteObesidad', 'AntecedenteVIH', 'personaid_count' 
   
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y AntecedenteObesidad
result_df = df_filtered.groupBy('Año', 'Mes','AntecedenteObesidad','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-------------------+--------------------+------------------+--------------------+
| Año|Mes|AntecedenteObesidad|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+-------------------+--------------------+------------------+--------------------+
|2022| 10|                 No|    54518 - Pamplona|                 1|                   1|
|2022|  4|                 No|       25785 - Tabio|                 1|                   3|
|2022|  4|                 No|    -1 - No Definido|               348|                 354|
|2022|  6|                 No| 68001 - Bucaramanga|                61|                  73|
|2022|  6|                 No|     15238 - Duitama|                 7|                  12|
|2022|  8|                 Sí|    05001 - Medellín|                12|                  13|
|2022|  6|                 No|23189 - Ciénaga D...|                 1|                   3|
|2022|  7|                 No|      68770 - Suaita|                 1|          

se verifican algunos resultados de enero, coinciden perfectamente

Año	Mes	AntecedenteObesidad	MunicipioAtencion	# Personas	# Atenciones
2022	enero	No	05129 - Caldas	177	230

Año	Mes	AntecedenteObesidad	MunicipioAtencion	# Personas	# Atenciones
2022	enero	No	05308 - Girardota	88	103


# 10. Agrupado por año, mes, AntecedenteVIH y MunicipioAtencion

In [72]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 'AntecedenteHipertension',
    'AntecedenteArtritis', 'AntecedenteAsma', 'AntecedenteCancer', 'AntecedenteDiabetes', 'AntecedenteEnfermedadHuerfana',
    'AntecedenteEPOC', 'AntecedenteERC', 'AntecedenteObesidad', 'AntecedenteVIH', 'personaid_count' 
   
]
df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y AntecedenteVIH
result_df = df_filtered.groupBy('Año', 'Mes','AntecedenteVIH','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+--------------+--------------------+------------------+--------------------+
| Año|Mes|AntecedenteVIH|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+--------------+--------------------+------------------+--------------------+
|2022| 10|            No|    54518 - Pamplona|                 1|                   1|
|2022|  4|            No|       25785 - Tabio|                 1|                   3|
|2022|  4|            No|    -1 - No Definido|               372|                 379|
|2022|  6|            No| 68001 - Bucaramanga|                72|                  89|
|2022|  6|            No|     15238 - Duitama|                 8|                  13|
|2022|  8|            Sí|    05001 - Medellín|                 1|                   1|
|2022|  6|            No|23189 - Ciénaga D...|                 1|                   3|
|2022|  7|            No|      68770 - Suaita|                 1|                   2|
|2022|  7|            No|       52001 - Pas

se comprueba los de enero y coincide exactamente con los del powerbi

Año	Mes	AntecedenteVIH	MunicipioAtencion	# Personas	# Atenciones
2022	enero	No	05129 - Caldas	195	256

Año	Mes	AntecedenteVIH	MunicipioAtencion	# Personas	# Atenciones
2022	enero	No	05308 - Girardota	106	128


# FACTORES (Pestaña recibida en PowerBI)

# 11. Agrupado por año, mes, FactorRiesgoCancer y MunicipioAtencion

In [74]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 
    
    'FactorRiesgoCancer','FactorRiesgoCondicionImpidaAislamiento' ,'FactorRiesgoDiabetes' ,'FactorRiesgoEnfermedadCardiovascular',
    'FactorRiesgoEnfermedadRespiratoriaCronica', 'FactorRiesgoInmunodeficiencia', 'FactorRiesgoMayor59' , 'personaid_count' 
   
]

df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y FactorRiesgoCancer
result_df = df_filtered.groupBy('Año', 'Mes','FactorRiesgoCancer','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+------------------+--------------------+------------------+--------------------+
| Año|Mes|FactorRiesgoCancer|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+------------------+--------------------+------------------+--------------------+
|2022| 10|                No|    54518 - Pamplona|                 1|                   1|
|2022|  4|                No|       25785 - Tabio|                 1|                   3|
|2022|  4|                No|    -1 - No Definido|               373|                 380|
|2022|  6|                No| 68001 - Bucaramanga|                71|                  89|
|2022|  6|                No|     15238 - Duitama|                 8|                  13|
|2022|  8|                Sí|    05001 - Medellín|                 3|                   3|
|2022|  6|                No|23189 - Ciénaga D...|                 1|                   3|
|2022|  7|                No|      68770 - Suaita|                 1|                   2|

Año	Mes	FactorRiesgoCancer	MunicipioAtencion	# Personas	# Atenciones
2022	enero	No	05308 - Girardota	107	129  

# 12. Agrupado por año, mes,FactorRiesgoCondicionImpidaAislamiento   y MunicipioAtencion

In [75]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 
    
    'FactorRiesgoCancer','FactorRiesgoCondicionImpidaAislamiento' ,'FactorRiesgoDiabetes' ,'FactorRiesgoEnfermedadCardiovascular',
    'FactorRiesgoEnfermedadRespiratoriaCronica', 'FactorRiesgoInmunodeficiencia', 'FactorRiesgoMayor59' , 'personaid_count' 
   
]

df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y FactorRiesgoCondicionImpidaAislamiento
result_df = df_filtered.groupBy('Año', 'Mes','FactorRiesgoCondicionImpidaAislamiento','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+--------------------------------------+--------------------+------------------+--------------------+
| Año|Mes|FactorRiesgoCondicionImpidaAislamiento|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+--------------------------------------+--------------------+------------------+--------------------+
|2022| 10|                                    No|    54518 - Pamplona|                 1|                   1|
|2022|  4|                                    No|       25785 - Tabio|                 1|                   3|
|2022|  4|                                    No|    -1 - No Definido|               373|                 380|
|2022|  6|                                    No| 68001 - Bucaramanga|                73|                  92|
|2022|  6|                                    No|     15238 - Duitama|                 8|                  13|
|2022|  6|                                    No|23189 - Ciénaga D...|                 1|                   3|
|

Año	Mes	FactorRiesgoCondicionImpidaAislamiento	MunicipioAtencion	# Personas	# Atenciones
2022	enero	No	05308 - Girardota	107	129

# 13 Agrupado por año, mes,FactorRiesgoDiabetes y MunicipioAtencion

In [76]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 
    
    'FactorRiesgoCancer','FactorRiesgoCondicionImpidaAislamiento' ,'FactorRiesgoDiabetes' ,'FactorRiesgoEnfermedadCardiovascular',
    'FactorRiesgoEnfermedadRespiratoriaCronica', 'FactorRiesgoInmunodeficiencia', 'FactorRiesgoMayor59' , 'personaid_count' 
   
]

df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y FactorRiesgoDiabetes
result_df = df_filtered.groupBy('Año', 'Mes','FactorRiesgoDiabetes','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+--------------------+--------------------+------------------+--------------------+
| Año|Mes|FactorRiesgoDiabetes|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+--------------------+--------------------+------------------+--------------------+
|2022| 10|                  No|    54518 - Pamplona|                 1|                   1|
|2022|  4|                  No|       25785 - Tabio|                 1|                   3|
|2022|  4|                  No|    -1 - No Definido|               372|                 378|
|2022|  6|                  No| 68001 - Bucaramanga|                69|                  88|
|2022|  8|                  Sí|    05001 - Medellín|                11|                  11|
|2022|  6|                  No|     15238 - Duitama|                 8|                  13|
|2022|  6|                  No|23189 - Ciénaga D...|                 1|                   3|
|2022|  7|                  No|      68770 - Suaita|                 1

# 14 Agrupado por año, mes, FactorRiesgoEnfermedadCardiovascular y MunicipioAtencion

In [77]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 
    
    'FactorRiesgoCancer','FactorRiesgoCondicionImpidaAislamiento' ,'FactorRiesgoDiabetes' ,'FactorRiesgoEnfermedadCardiovascular',
    'FactorRiesgoEnfermedadRespiratoriaCronica', 'FactorRiesgoInmunodeficiencia', 'FactorRiesgoMayor59' , 'personaid_count' 
   
]

df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y FactorRiesgoEnfermedadCardiovascular
result_df = df_filtered.groupBy('Año', 'Mes','FactorRiesgoEnfermedadCardiovascular','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+------------------------------------+--------------------+------------------+--------------------+
| Año|Mes|FactorRiesgoEnfermedadCardiovascular|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+------------------------------------+--------------------+------------------+--------------------+
|2022|  6|                                  No| 68001 - Bucaramanga|                65|                  78|
|2022|  4|                                  No|    -1 - No Definido|               372|                 378|
|2022|  8|                                  Sí|    05001 - Medellín|                41|                  41|
|2022|  2|                                  Sí|      23417 - Lorica|                 1|                   1|
|2022|  9|                                  No|    -1 - No Definido|               335|                 337|
|2022|  7|                                  No|       52001 - Pasto|                69|                  88|
|2022| 11|         

# 15 Agrupado por año, mes, FactorRiesgoEnfermedadRespiratoriaCronica y MunicipioAtencion

In [78]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 
    
    'FactorRiesgoCancer','FactorRiesgoCondicionImpidaAislamiento' ,'FactorRiesgoDiabetes' ,'FactorRiesgoEnfermedadCardiovascular',
    'FactorRiesgoEnfermedadRespiratoriaCronica', 'FactorRiesgoInmunodeficiencia', 'FactorRiesgoMayor59' , 'personaid_count' 
   
]

df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y FactorRiesgoEnfermedadRespiratoriaCronica
result_df = df_filtered.groupBy('Año', 'Mes','FactorRiesgoEnfermedadRespiratoriaCronica','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-----------------------------------------+--------------------+------------------+--------------------+
| Año|Mes|FactorRiesgoEnfermedadRespiratoriaCronica|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------------------------------------+--------------------+------------------+--------------------+
|2022|  6|                                       No| 68001 - Bucaramanga|                72|                  91|
|2022|  4|                                       No|    -1 - No Definido|               373|                 380|
|2022|  8|                                       Sí|    05001 - Medellín|                 4|                   4|
|2022|  6|                                       No|     15238 - Duitama|                 8|                  13|
|2022|  9|                                       No|    -1 - No Definido|               335|                 337|
|2022|  7|                                       No|       52001 - Pasto|               

# 16 Agrupado por año, mes,FactorRiesgoInmunodeficiencia y MunicipioAtencion

In [79]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 
    
    'FactorRiesgoCancer','FactorRiesgoCondicionImpidaAislamiento' ,'FactorRiesgoDiabetes' ,'FactorRiesgoEnfermedadCardiovascular',
    'FactorRiesgoEnfermedadRespiratoriaCronica', 'FactorRiesgoInmunodeficiencia', 'FactorRiesgoMayor59' , 'personaid_count' 
   
]

df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y FactorRiesgoInmunodeficiencia
result_df = df_filtered.groupBy('Año', 'Mes','FactorRiesgoInmunodeficiencia','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-----------------------------+--------------------+------------------+--------------------+
| Año|Mes|FactorRiesgoInmunodeficiencia|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+-----------------------------+--------------------+------------------+--------------------+
|2022| 10|                           No|    54518 - Pamplona|                 1|                   1|
|2022|  4|                           No|       25785 - Tabio|                 1|                   3|
|2022|  4|                           No|    -1 - No Definido|               373|                 380|
|2022|  6|                           No| 68001 - Bucaramanga|                72|                  89|
|2022|  6|                           No|     15238 - Duitama|                 8|                  13|
|2022|  8|                           Sí|    05001 - Medellín|                 1|                   1|
|2022|  6|                           No|23189 - Ciénaga D...|                 1|  

# 17 Agrupado por año, mes,FactorRiesgoMayor59 y MunicipioAtencion

In [80]:
# Seleccionar solo las columnas deseadas
columns_to_keep = [
    'PersonaBasicaID', 'FechaRegistro', 'MunicipioAtencion', 
    
    'FactorRiesgoCancer','FactorRiesgoCondicionImpidaAislamiento' ,'FactorRiesgoDiabetes' ,'FactorRiesgoEnfermedadCardiovascular',
    'FactorRiesgoEnfermedadRespiratoriaCronica', 'FactorRiesgoInmunodeficiencia', 'FactorRiesgoMayor59' , 'personaid_count' 
   
]

df_filtered = df2.select(*columns_to_keep)

# Convertir 'FechaRegistro' a tipo fecha con el formato correcto
df_filtered = df_filtered.withColumn('FechaRegistro', F.to_date('FechaRegistro', 'yyyy-MM-dd'))


# Extraer año y mes de la columna 'fechaid'
df_filtered = df_filtered.withColumn('Año', F.year('FechaRegistro'))
df_filtered = df_filtered.withColumn('Mes', F.month('FechaRegistro'))

# Agrupar por año, mes y FactorRiesgoMayor59
result_df = df_filtered.groupBy('Año', 'Mes','FactorRiesgoMayor59','MunicipioAtencion').agg(
    F.countDistinct('PersonaBasicaID').alias('Número_de_Personas'),
    F.sum('personaid_count').alias('Número_de_Atenciones')
)

# Mostrar los resultados
result_df.show(100)

# Imprimir los nombres de las columnas agrupadas para verificar
print("Nombres de las columnas agrupadas:")
print(result_df.columns)

+----+---+-------------------+--------------------+------------------+--------------------+
| Año|Mes|FactorRiesgoMayor59|   MunicipioAtencion|Número_de_Personas|Número_de_Atenciones|
+----+---+-------------------+--------------------+------------------+--------------------+
|2022|  4|                 No|       25785 - Tabio|                 1|                   3|
|2022|  4|                 No|    -1 - No Definido|               370|                 376|
|2022|  8|                 Sí|    05001 - Medellín|                61|                  63|
|2022|  6|                 No| 68001 - Bucaramanga|                50|                  62|
|2022|  6|                 No|     15238 - Duitama|                 4|                   7|
|2022|  2|                 Sí|      23417 - Lorica|                 6|                   6|
|2022|  3|                 Sí|     08433 - Malambo|                 1|                   1|
|2022|  7|                 Sí| 68547 - Piedecuesta|                 2|          

In [82]:
spark.stop()